
## Questions API Client
* What about the `base_endpoint` URL? Could/should this be added as a class variable? Or should it be added within the `__init__` method? When do you choose one or the other? Why not just put all of these inside the `__init__` function? Just seems redundant somehow...
* To make our Client more robust, should we add `assert` or `raise Exception` in some places? Is that common for `class` objects?
* Do you like to use constants? `API_KEY` vs. `api_key`?
* Better understanding `location` and the `extract_latlng()` method. Okay, so when we init a new instance, we're setting `self.location_query = address_or_postal_code`. Even if an address is passed during the initialization, we're going to set the `self.location_query` and `self.lat`, `self.lng` values. **However**, if we *do not* pass an `address_or_postal_code` during initialization, then the `self.lat` and `self.lng` values will be their defaults `None`. Then later on a user may also choose to update the address/location via the `search()` method. We're giving them the option to pass a new `location` argument to `search(self, location)` method. If a new `location` is provided, then we re-run the `extract_latlng()` method but now passing it this updated `location` value. However, before we do any of that, we first (within the `search()` method) set some *local* function vars `lat, lng = self.lat, self.lng` since we want to first get whatever those instance values are currently before we run the `extract_latlng()` method to get updated values. This means that if a `location` has indeed been passed to `extract_latlng()` (any time *after* the instance has been initialized), then we're going to update the `self.location_query`, `self.lat`, and `self.lng` class variable values.
**It seems to be a difference of referencing the `class` variable value (`self.lat`) vs. the `local` variable value (`lat`).** Recall that originally we have `params["address"]: self.location_query`, which during initialization could be `None` (if `address_or_postal_code is None`). 
* What's the difference between `self.lat = lat` vs. `lat = self.lat`?

## Google Maps API Client (or, Client API)
### Geocoding
* First thing we need is the extracted `lat` and `lng` values
* Move `data_type='json'` as a class variable so every request we make uses `json`.
* Same with the `address` (or location). That's going to come with every request so can just pull it out and make it into a class variable, e.g., `location_query: t.Optional[str] = None`
* We can set `lat` and `lng` values back to the class itself using `self.lat = lat` and `self.lng = lng`. Otherwise, they'll remain as `None` as per their defaults. This allows us to use the `self.lat` and `self.lng` values later within other class methods, etc. 
* In fact, after these changes we could actually initialize this Client (class) this way. Meaning we can define the `__init__` function and have it require an `address_or_postal_code` argument, which can then be used to extract the `lat` and `lng` values by doing `self.extract_latlng()` as part of the instance initialization. We can have `address_or_postal_code` be an *optional* argument by defaulting to `None`. This means that the user can just create a new instance of the Client without having to also specify an `address` from the beginning. However, if the user does provide/pass a `address_or_postal_code`, then we could handle/update the `lat` and `lng` values based on the provided location.
* Same goes for `api_key` as we must have one. Therefore let's add it as a class variable and default to `None`. Then let's `raise Exception` if one isn't passed as an argument.
### Places Nearby Search
* Going to just implement the nearby searches feature instead of just single `findplacefromtext`.
* We could consider giving the user the chance to change the `lat` and `lng` values when using the `search()` class method we're building. To do this, we can add a `location=None` parameter to our `search(self, keyword="Tex-Mex food", location=None)` definition. So if a `location` is passed, then we'd have to update the `lat`, `lng` values using our `extract_latlng()` method. However, we want to reset/update the values based on `location`, so it makes sense to add `location=None` to the `extract_latlng(self, location=None)` method as well.
    * ??         # NOTE ?? - Why not just set self.location_query = location?
### Places Detail
* More straight-forward since we don't mess with `location', only `place_id` and `fields`.


In [11]:
import typing as t
from urllib.parse import urlencode, urlparse, parse_qsl

import requests

from google_geocoding_places_api import settings

In [12]:
API_KEY = settings.API_KEY

In [13]:
class GoogleMapsAPIClient(object):
    lat: t.Optional[float] = None
    lng: t.Optional[float] = None
    data_type: str = "json"  # or lookup_type
    location_query: t.Optional[str] = None
    api_key: t.Optional[str] = None

    def __init__(self, api_key: t.Optional[str] = None, address_or_postal_code: t.Optional[str] = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key is None:
            raise Exception("Must provide an API key.")
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query is not None:
            self.extract_latlng()

    # We have our 'url' ready for the request but we want to return the latitude and longitude
    # Let's convert this into a function
    def extract_latlng(self, location: t.Optional[str] = None) -> t.Tuple:
        # set 'loc_query' to original default self.location_query set from instantiation
        loc_query: str = self.location_query  
        # If 'location' is passed/provided, then update self.location_query
        if location is not None:
        # NOTE ?? - Why not just set self.location_query = location?
            loc_query = location
        
        # outputFormat
        params: t.Dict = {"address": loc_query, "key": self.api_key}
        url_params: str = urlencode(params)
        endpoint: str = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        url: str = f"{endpoint}?{url_params}"

        # Now we have our request url let's make the request
        r = requests.get(url)
        if r.status_code not in range(200, 299):
            print(f"Request failed! {url}")
            return {}
        
        # Try to set/extract the lat lng data from 'location' key 
        # r.json()['results'][0]['geometry']['location']
        # {'lat': 37.4220578, 'lng': -122.0840897}
        latlng: t.Dict[str, float] = {}
        try:
            latlng = r.json()['results'][0]['geometry']['location']
        except:
            pass

        # Set these lat,lng values back to the class variables
        lat: float = latlng.get('lat')
        lng: float = latlng.get('lng')
        self.lat = lat
        self.lng = lng

        # Use .get('key') to get result or None. If I use d['key'] could get KetError
        # return latlng.get('lat'), latlng.get('lng')  # (None, None)
        return lat, lng


    # Let's try the Nearby Search request to get multiple results
    def search(self, keyword: str = "Tex-Mex food", location: t.Optional[str] = None, radius: int = 1000) -> t.Dict:
        # https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
        # Reference the class variables first:
        lat: float
        lng: float
        lat, lng = self.lat, self.lng
        # Check if location was passed
        if location is not None:
            # Updated lat, lng values by extracting
            # ?? Don't we have to pass 'location' to this extract_latlng() fn? YES!
            lat, lng = self.extract_latlng(location=location)  
        base_endpoint: str = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params: t.Dict = {
            "key": self.api_key,
            "location": f"{lat},{lng}", # reference local 'lat' instead of class self.lat
            "radius": radius,  # meters
            "keyword": keyword
        }
        params_encoded: str = urlencode(params)
        places_endpoint: str = f"{base_endpoint}?{params_encoded}"

        r = requests.get(places_endpoint)
        if r.status_code not in range(200, 299):
            print(f"Search request failed: {places_endpoint}")
            return {}
        return r.json()  # Could refine later


    def detail(self, place_id: str = "ChIJ8ZTVZJmwj4ARQFv0RXspg3A", fields: t.List[str] = ["name", "rating", "formatted_phone_number", "formatted_address"]) -> t.Dict:
        # Once we get our place either from Place or Nearby request we can
        # use Place Detail along with place_id for a detail_lookup_request
        # https://maps.googleapis.com/maps/api/place/details/json?place_id=ChIJN1t_tDeuEmsRUsoyG83frY4&fields=name,rating,formatted_phone_number&key=YOUR_API_KEY
        base_details_endpoint: str = f"https://maps.googleapis.com/maps/api/place/details/{self.data_type}"
        params: t.Dict = {
            "place_id": f"{place_id}",
            "fields": ",".join(fields),
            "key": self.api_key
        }
        params_encoded: str = urlencode(params)
        details_endpoint: str = f"{base_details_endpoint}?{params_encoded}"

        r = requests.get(details_endpoint)
        if r.status_code not in range(200, 299):
            print(f"Detail request failed: {details_endpoint}")
            return {}
        return r.json()  # Could refine later
    

In [14]:
client = GoogleMapsAPIClient(api_key=API_KEY, address_or_postal_code="Austin, TX")

# print(client.lat, client.lng)  # 30.267153 -97.7430608

In [15]:
# Let's test out some search() and detail()
client.search("Tacos", location="Georgetown, TX")  # works!

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 30.6406001, 'lng': -97.6783354},
    'viewport': {'northeast': {'lat': 30.64204402989272,
      'lng': -97.67697072010726},
     'southwest': {'lat': 30.63934437010727, 'lng': -97.67967037989271}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
   'name': 'El Charrito',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117799068673281855180">A Google User</a>'],
     'photo_reference': 'CmRaAAAAgrAN2z8JrKVbJG6B2xWhI4f65j6nJydz6nu0Y0lSQdorpqeulkXyAVxBv5gxtZBROOTbgtfWlZnfqSLTsAfGWQqQA-5AL0J47zo9OyOCtr_PpseDiX-sa0j-3Vly4TUIEhC5RMlH48oOjhxELeN_qFHMGhSUdBnPZ_DKuwbUYeI9SwupIrQrKw',
     'width': 4032}],
   'place_id': 'ChIJD2lEIWrWRIYRIHwDUMk--8U',
   'plus_code': {'compound_code': 'J8RC+6M Georgetown, Texas',
    'global_code': '8624J8RC+6M'},
   'price_

In [16]:
# Dos Salsas 'ChIJl4bD7WfWRIYRj928bZAMexI'
client.detail(place_id='ChIJl4bD7WfWRIYRj928bZAMexI')  # works!

{'html_attributions': [],
 'result': {'formatted_address': '1104 S Main St, Georgetown, TX 78626, USA',
  'formatted_phone_number': '(512) 930-2343',
  'name': 'Dos Salsas',
  'rating': 4.3},
 'status': 'OK'}